In [130]:
import pandas as pd
import numpy as np
from scipy import stats 
import operator

# Summary
In this problem, we looked at classifying protein sequences according to whether they belong to a particular protein family or not. For this task, we used the string kernel, as well as a modiﬁed version of this kernel. The ﬁles pa4train.txt and pa4test.txt are already downloaded.
## Part 1

In [133]:
df_1

,train,test
3,0.0134986,0.0422164
4,0.00964187,0.0356201
5,0.0101928,0.0646438


## Part 2

In [175]:
df_2

,train,test
3,0.0132231,0.0422164
4,0.00964187,0.0356201
5,0.0101928,0.0646438


## Part 3
Find the two coordinates in w with the highest positive values. 
What are the substrings corresponding to these coordinates? 

(at the count of 3)
* 'WDTAG'
* 'DTAGQ'
* 'LFLNK'
* 'GKSSL'
* 'KVGPD'

In [3]:
tr_x = []
tr_y = []
f = open("pa4train.txt", "r")
for x in f:
    tr_x.append(x[:-4])
    tr_y.append(int(x[-3:-1]))
tr_x[0], tr_y[0]

('TFGAGEADCGLRPLFEKKQVQDQTEKELFESYIEGRXIVEGQDAEVGLSPWQVMLFRKSPQELLCGASLISDRWVLTAAHCLLYPPWDKNFTVDDLLVRIGKHSRTRYERKVEKISMLDKIYIHPRYNWKENLDRDIALLKLKRPIELSDYIHPVCLPDKQTAAKLLHAGFKGRVTGWGNRRETWTTSVAEVQPSVLQVVNLPLVERPVCKASTRIRITDNMFCAGYKPGEGKRGDACEGDSGGPFVMKSPYNNRWYQMGIVSWGEGCDRDGKYGFYTHVFRLKKWIQKVIDRLGS',
 -1)

In [4]:
te_x = []
te_y = []
f = open("pa4test.txt", "r")
for x in f:
    te_x.append(x[:-4])
    te_y.append(int(x[-3:-1]))
te_x[0], te_y[0]

('DPTQLSPRPPIVTIMGHVDHGKTSLLDAIRQTNTAAKEFGGITQKIGAYQVKNQEGKTITFIDTPGHEAFTGMRARGAQVTDIVVLVVAGDDGLKQQTEEAISHAKSAKTPIIVFINKMDKPTANPDMVIQQLNKFDLVPEEWGGDTIFVKGSALTKEGIQELLDSILLVAEVEDYKANFNAHSSGY',
 1)

## Part 1
First, we will use the string kernel function for our kernel. Recall from class that given two strings s and t, the string kernel Kp(s,t) is the number of substrings of length p that are common to both s and t, where a string that occurs a times in s and b times in t is counted ab times. For this problem, use p = 3, p = 4 and p = 5. Write down the training and test errors of kernel perceptron for p = 3,4,5 on this dataset. 

In [165]:
class KP():
    def __init__(self, p):
        self.p = p
        self.x = None
        self.y = None
        self.tr_counts = None
        self.te_counts = None
        
    def count(self, tr_x, te_x):
        self.tr_x = tr_x
        tr_counts = []
        tr_num = len(tr_x)
        p = self.p
        for i in range(tr_num):
            dict = {}
            str = tr_x[i]
            for j in range(len(str) - p + 1):
                sub = str[j:j+p]
                if sub in dict.keys():
                    dict[sub] += 1
                else:
                    dict[sub] = 1
            tr_counts.append(dict)
        self.tr_counts = tr_counts
    
        if tr_x == te_x:
            self.te_counts = self.tr_counts
        
        else:
            self.te_x = te_x
            te_counts = []
            te_num = len(te_x)
            for i in range(te_num):
                dict = {}
                str = te_x[i]
                for j in range(len(str) - p + 1):
                    sub = str[j:j+p]
                    if sub in dict.keys():
                        dict[sub] += 1
                    else:
                        dict[sub] = 1
                te_counts.append(dict)
            self.te_counts = te_counts
            
            
    def similarity(self, i, j, bool):
        total = 0
        if bool:
            s = self.tr_counts[i]
        else:
            s = self.te_counts[i]
        t = self.tr_counts[j]
        for key in s.keys():
            if key in t.keys():
                total += s[key] * t[key]
        return total
    
    def predict(self, i, bool):
        res = 0
        for j in np.where(self.err == 1)[0]:
            res += self.tr_y[j] * self.similarity(i, j, bool)
        return res
    
    def train(self, y):
        self.tr_y = y
        num = len(y)
        self.err = np.zeros(num)
        for i in range(num):
            if y[i] * self.predict(i, True) <= 0:
                self.err[i] = 1
                
    def score(self, te_y):
        loss = 0
        num = len(te_y)
        for i in range(num):
            if te_y[i] * self.predict(i, False) <= 0:
                loss += 1
        return loss/len(te_y)

In [166]:
kp = KP(p=2)
kp.count(tr_x, tr_x)
kp.train(tr_y)
kp.score(tr_y)

0.07134986225895316

In [111]:
df_1 = pd.DataFrame(columns=['train', 'test'])
for p in [3, 4, 5]:
    kp_tr = KP(p=p)
    kp_tr.count(tr_x, tr_x)
    kp_tr.train(tr_y)
    df_1.loc[p, 'train'] = kp_tr.score(tr_y)
    kp_te = KP(p=p)
    kp_te.count(tr_x, te_x)
    kp_te.train(tr_y)
    df_1.loc[p, 'test'] = kp_te.score(te_y)
df_1

,train,test
3,0.0134986,0.0422164
4,0.00964187,0.0356201
5,0.0101928,0.0646438


## Part 2
Next, repeat Part (1) with a slight modiﬁcation of the string kernel, Mp(s,t). Given two strings s and t, the modiﬁed string kernel Mp(s,t) is the number of substrings of length p that are common to both s and t, where a string that occurs a times in s and b times in t is counted only once. What are the training and test errors for this kernel for p = 3,4,5? 

In [173]:
class KP_2():
    def __init__(self, p):
        self.p = p
        self.x = None
        self.y = None
        self.tr_counts = None
        self.te_counts = None
        
    def count(self, tr_x, te_x):
        self.tr_x = tr_x
        tr_counts = []
        tr_num = len(tr_x)
        p = self.p
        for i in range(tr_num):
            dict = []
            str = tr_x[i]
            for j in range(len(str) - p + 1):
                sub = str[j:j+p]
                if sub not in dict:
                    dict.append(sub)
            tr_counts.append(dict)
        self.tr_counts = tr_counts
    
        if len(tr_x) == len(te_x):
            self.te_counts = self.tr_counts
        
        else:
            self.te_x = te_x
            te_counts = []
            te_num = len(te_x)
            for i in range(te_num):
                dict = []
                str = te_x[i]
                for j in range(len(str) - p + 1):
                    sub = str[j:j+p]
                    if sub not in dict:
                        dict.append(sub)
                te_counts.append(dict)
            self.te_counts = te_counts
            
            
    def similarity(self, i, j, bool):
        total = 0
        if bool:
            s = self.tr_counts[i]
        else:
            s = self.te_counts[i]
        t = self.tr_counts[j]
        return len(set(s)&set(t))
    
    def predict(self, i, bool):
        res = 0
        for j in np.where(self.err == 1)[0]:
            res += self.tr_y[j] * self.similarity(i, j, bool)
        return res
    
    def train(self, y):
        self.tr_y = y
        num = len(y)
        # err is a list of a size of the training data
        self.err = np.zeros(num)
        for i in range(num):
            if y[i] * self.predict(i, True) <= 0:
                # whenever there is an error, i update the err with 1, err is like w
                self.err[i] = 1
                
    def score(self, te_y):
        loss = 0
        num = len(te_y)
        for i in range(num):
            if te_y[i] * self.predict(i, False) <= 0:
                loss += 1
        return loss/len(te_y)

In [174]:
df_2 = pd.DataFrame(columns=['train', 'test'])
for p in [3, 4, 5]:
    kp_tr = KP_2(p=p)
    kp_tr.count(tr_x, tr_x)
    kp_tr.train(tr_y)
    df_2.loc[p, 'train'] = kp_tr.score(tr_y)
    kp_te = KP(p=p)
    kp_te.count(tr_x, te_x)
    kp_te.train(tr_y)
    df_2.loc[p, 'test'] = kp_te.score(te_y)
df_2

,train,test
3,0.0132231,0.0422164
4,0.00964187,0.0356201
5,0.0101928,0.0646438


## Part 3
Find the two coordinates in w with the highest positive values. You should be able to do this without explicitly computing all the coordinates of w. What are the substrings corresponding to these coordinates? These coordinates correspond to those substrings whose presence most strongly indicates that the protein belongs in the family.

In [155]:
class KP_3():
    def __init__(self, p):
        self.p = p
        self.x = None
        self.y = None
        self.tr_counts = None
        self.te_counts = None
        
    def count(self, tr_x):
        self.tr_x = tr_x
        tr_counts = []
        tr_num = len(tr_x)
        p = self.p
        for i in range(tr_num):
            dict = {}
            str = tr_x[i]
            for j in range(len(str) - p + 1):
                sub = str[j:j+p]
                if sub in dict.keys():
                    dict[sub] += 1
                else:
                    dict[sub] = 1
            tr_counts.append(dict)
        self.tr_counts = tr_counts
        return self.tr_counts   
            
    def similarity(self, i, j, bool):
        total = 0
        if bool:
            s = self.tr_counts[i]
        else:
            s = self.te_counts[i]
        t = self.tr_counts[j]
        for key in s.keys():
            if key in t.keys():
                total += s[key] * t[key]
        return total
    
    def predict(self, i, bool):
        res = 0
        for j in np.where(self.err == 1)[0]:
            res += self.tr_y[j] * self.similarity(i, j, bool)
        return res
    
    def train(self, y):
        self.tr_y = y
        num = len(y)
        self.err = np.zeros(num)
        for i in range(num):
            if y[i] * self.predict(i, True) <= 0:
                self.err[i] = 1
        return self.err

In [156]:
kp = KP_3(p=5)
count = kp.count(tr_x)
err = kp.train(tr_y)

In [169]:
strings = {}
for i in range(len(err)):
    # if it is a misclassified row
    if err[i] == 1.:
        # get the label
        y = tr_y[i]
        # get the dinctionary 
        dict = count[i]
        for j in dict.keys():
            if j in strings.keys():
                strings[j] += y * dict[j]
            else:
                strings[j] = y * dict[j]

In [170]:
listofTuples = sorted(strings.items() , reverse=True, key=lambda x: x[1])
listofTuples[:10]

[('WDTAG', 3),
 ('DTAGQ', 3),
 ('LFLNK', 3),
 ('GKSSL', 3),
 ('KVGPD', 3),
 ('TAGQE', 2),
 ('YYRGA', 2),
 ('NKCDM', 2),
 ('TVKLL', 2),
 ('KLLLL', 2)]

### Part 3
Find the two coordinates in w with the highest positive values. 
What are the substrings corresponding to these coordinates? 
* 'WDTAG'
* 'DTAGQ'
* 'LFLNK'
* 'GKSSL'
* 'KVGPD'